#### **Entrez, BLAST and Multiple Sequence Alignments**
###### **Alliance meeting Paraguay EUSAT-RCS consortium, 2024**

###### *Designed by Jorge G. García, 2024*
---
Welcome to a brief introduction into comparative genomics. Specialized database queries and sequence alignments form the backbone of innumerable research projects across the world, and the fight against TB makes extensive use of these tools. Some among the uses of these technologies are:
- Identifying point mutations in selected genes across different strains of Mycobacterium tuberculosis (Mtb).
- Tracking the natural history of Mtb across regions and time periods.
- Revealing patterns of spread and transmission through phylogenetic analysis.
- Discovering potential targets for new drugs by comparing gene sequences between drug-resistant and drug-susceptible strains.
- Assessing the genetic diversity of Mtb strains to inform vaccine development and tailor treatments to specific variants.

As we embark on this journey, we'll explore how to programmatically retrieve DNA sequences from specialized databases and how to compare them in insightful ways. This workshop is designed to equip you with the knowledge and tools to contribute to the global effort against TB through the lens of bioinformatics. Even though we will use real-word, up-to-date data throughout most of the exercise, it is directed towards a wide audience and doesn't require any previous knowledge of bioinformatic pipelines, or even Python! 

If you have no experience with code whatsoever, fret not! This exercise is designed to lead you effortlessly straight to the end. If for some reason you break it, restart and voilà. If you are already familiar with programming, this exercise should be of use to understand how biological data is retrieved and aligned. Feel free to tinker with other parameters and explore how the code works!

Last but not least, across the exercise you will see many hyperlinks scattered across the sections. Click on them to access extra resources that will enrich your experience, add context or reveal interesting facts about the lesson.

With all this in mind, take a deep breath and press the Run button in the first cell. Welcome to bioinformatics.


In [59]:
from Bio import Entrez, SeqIO, Phylo, AlignIO, Align
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import pandas as pd
import numpy as np
import time

Congratulations! You just imported the necessary libraries to perform all the analysis in this exercise. 

In programming, a library is a collection of precompiled routines - sets of instructions - that a program can use. These routines are designed to accomplish specific tasks, such as handling files and data, performing mathematical computations, or managing network connections - and that is exactly what we are going to do with these. Many of these libraries we will be using, like pandas for tabular data management or the powerful numpy for mathematical operations, are used everywhere across the world in many different fields, from finance to weather forecasting. 

You will also see that many of our import statements start with "Bio" - short of [Biopython](https://biopython.org/), a specialized and comprehensive library to perform operations on biological data. 

But before proceeding, we should have very clear what we are looking for. [Mycobacterium tuberculosis](https://www.microbiologyresearch.org/content/journal/micro/10.1099/mic.0.000601) genome is approximately 4.4 million bases in length, encoding around 4000 genes and innumerable non-codifying - but still important - regions. As researchers, we are often interested in specific segments of this whole, particularly those associated to traits in the pathogen, and one the most noteworthy ones is antibiotic resistance.

There is ample evidence of the association between the activation of certain genes in Mtb and antibiotic resistance. Some of the most prominent are:
- [**rpoB**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8502021/): Mutations in the *rpoB* gene, which encodes the β subunit of RNA polymerase, are responsible for resistance to rifampicin, a key first-line drug for TB treatment. Resistance to rifampicin is a critical marker for multidrug-resistant TB (MDR-TB).

- [**katG**](https://journals.asm.org/doi/10.1128/jb.175.13.4255-4259.1993): Mutations in this gene, which encodes catalase-peroxidase, are associated with isoniazid resistance. Isoniazid, another first-line anti-TB drug, requires activation by KatG to exert its antibacterial effect. Mutations in katG can lead to a loss of this activation, resulting in resistance.

- [**inhA**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7856099/#:~:text=KatG%20and%20inhA%20gene%20mutations%20are%20the%20main,are%20activated%20by%20monooxygenase%20EthA%20and%20catalase-peroxidase%20KatG.): This gene encodes an enzyme involved in the synthesis of mycolic acid, a component of the Mtb cell wall. Mutations in the promoter region of *inhA* can also confer resistance to isoniazid.

- [**embB**](https://pubmed.ncbi.nlm.nih.gov/33210572/#:~:text=Ethambutol%20%28EMB%29%20is%20one%20of%20the%20first-line%20drugs,for%20around%2070%25%20clinical%20EMB%20resistant%20M.%20tuberculosis.): The *embB* gene is associated with resistance to ethambutol, a first-line drug that targets cell wall synthesis. Mutations in embB can affect the drug's ability to inhibit the arabinogalactan synthesis, leading to resistance.

We will look into the first one, *rpoB*, and its role in rifampicine resistance development. [Rifampicine](https://go.drugbank.com/drugs/DB01045) is a widely used antibiotic used in TB treatment, and works by inhibiting the RNA polymerase enzyme, which is crucial for RNA synthesis. When used in combination therapy, rifampicin helps prevent the development of drug-resistant TB strains. However, monotherapy or improper use of rifampicin can lead to the emergence of resistance, which is a growing concern worldwide. 

You will now perform a simulacrum of a search against the Entrez database, looking for any publication that contains nucleotide sequences (DNA in our case) and the words "Mycobacterium", "tuberculosis" and "rifampicin" in its title. Press the Run button in the next cell and see what happens!


[^1]: test


In [60]:
def fetch_and_filter_sequences():
    seq_length = 200
    print('Fetching sequences...')
    search_handle = Entrez.esearch(db="nucleotide", term="Mycobacterium tuberculosis rifampicin[Title]", retmax=100)
    search_results = Entrez.read(search_handle)
    search_handle.close()
    id_list = search_results['IdList']

    fetch_handle = Entrez.efetch(db="nucleotide", id=id_list, rettype="fasta", retmode="text")
    sequences = list(SeqIO.parse(fetch_handle, "fasta"))
    fetch_handle.close()

    filtered_sequences = [seq for seq in sequences if len(seq.seq) <= seq_length]
    SeqIO.write(filtered_sequences, "filtered_sequences.fasta", "fasta")
    print(f"Filtered {len(filtered_sequences)} sequences at most {seq_length}bp.")

def fetch_and_filter_sequences_sim():
    seq_length = 200
    print('Fetching sequences...')
    with open('filtered_sequences.fasta', 'r') as file:
        filtered_sequences = list(SeqIO.parse(file, 'fasta'))
        time.sleep(5)
        print(f"Filtered {len(filtered_sequences)} sequences at most {seq_length}bp.")


fetch_and_filter_sequences_sim()

Fetching sequences...


Filtered 2 sequences at most 200bp.


As you can see, the previous cell contains two functions, called almost exactly the same. Well, there's no mistery to it, the first one is the real deal, and the second a simple simulacrum that outputs the same messages. Why are we doing this? Aren't we supposed to retrieve real data in real time? Well, there are two reasons for this:
1. This operation runs in the background, and depending on the complexity of the query, it may take several minutes of you looking at an empty terminal before it finishes.
2. It takes computational resources out of the Entrez database to perform a query. We can save them this redundant effort by having the files already prepared and ready to go, just as you do!

You are however, very welcome to check the exact same file it would have downloaded otherwise, named "aligned_sequences.fasta". A [FASTA](https://www.ncbi.nlm.nih.gov/genbank/fastaformat/) file is a plain text document that usually contains a header and a given sequence. For example, our FASTA file contains two sequences, the first one being:

```
>LN651305.1 Mycobacterium tuberculosis partial rpoB gene, isolate Kw9101-13, rifampicin resistance determining region
GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATGTACCAG
AACAACCCGCTGTCGGGGTTGACCCACAAGCGCCGACTGTCGGCGCTGGGGCCCGGCGGT
CTGTCACGTGAGCGTGCCGGGCTG
```

FASTA file entries always start with a header. Most of the structure of this header beyond the first ">" symbol is not a hard requirement of FASTA files - we could change it and our script would work just as fine - but it contains useful information for us:
1. Sequence ID, an exclusive identifier of the sequence.
2. The source organism.
3. The location of the sequence.
4. The rol of the sequence in the genome.

However, FASTA files are widely used not for their headers, but for their sequences. They are a light, straightforward way to share sequences, be it proteins or nucleic acids. The Biopython library contains routines to parse and extract structured information from these files in a convenient way for further exploration. Press the Run button in the next cell to continue.

In [61]:
def seq_reader():
    with open('filtered_sequences.fasta', 'r') as file:
        sequences = list(SeqIO.parse(file, 'fasta'))
        print(f"{len(sequences)} sequences read.")
        return sequences
    
seq_reader()

2 sequences read.


[SeqRecord(seq=Seq('GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATG...CTG'), id='LN651305.1', name='LN651305.1', description='LN651305.1 Mycobacterium tuberculosis partial rpoB gene, isolate Kw9101-13, rifampicin resistance determining region', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATG...CTG'), id='LN651304.1', name='LN651304.1', description='LN651304.1 Mycobacterium tuberculosis partial rpoB gene, isolate Kw5741-13, rifampicin resistance determining region', dbxrefs=[])]

Now this is much better! We can see all the information retrieved by categories. But then one wonders, what is this database I just obtained this data from? Why are their descriptions so similar? Should I trust it? 

Great questions! Let's explore for a few seconds the world of biological databases and the place Entrez has among them.

The [Entrez](https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html) database system is a comprehensive and integrated suite of databases developed and maintained by the [National Center for Biotechnology Information (NCBI)](https://www.ncbi.nlm.nih.gov/). It provides access to a wide range of biological data, including nucleotide and protein sequences, genome data, chemical compounds, literature references, and much more. The Entrez system is designed to facilitate the search and retrieval of biological information across various databases, making it a crucial resource for researchers in the fields of molecular biology, genetics, medicine, and bioinformatics. 

TL;DR: Look at it like a sort of Google for biochemical data. Its integrated nature allows for a seamless exploration of related data across different biological aspects, making it an essential resource for researchers around the world.

Our function merely "asked" the Nucleotide database, part of Entrez, for any entry matching our criteria, and returned a collection of sequences. Then, those sequences with a length equal or less than 200 base pairs were selected to continue the analysis downstream. Neither the database nor our function know what we plan to do with these sequences so it is up to us to analyse them, starting with verifying whether they are identical or not. Press the Run button in the next cell to continue.

In [62]:
def seq_identical():
    sequence_info = seq_reader()
    sequence_holder = list(np.zeros(len(sequence_info), dtype=int))
    for i, seq in enumerate(sequence_info):
        sequence_holder[i] = seq.seq

    all_identical = set(tuple(seq) for seq in sequence_holder) == 1
    
    if all_identical:
        print('All sequences are identical.')
    else:
        print('Sequences are not identical.')

seq_identical()

2 sequences read.
Sequences are not identical.


Turns out they are not! This is just a simple and fast way to verify if an indeterminate number of sequences are the exact same - in content and in order - without having to check every possible pairwise comparison.

**Extra-challenge for Python lovers** -> Create and run another function that fulfills the same goal <u>without performing pairwise comparisons</u>.

We can continue comparing these sequences. In fact, we are going to take advantage of them being just two. It's time to introduce pairwise sequence alignment!

Imagine the following two sequences:
- *Sequence A*: `ACTGTCGCA`
- *Sequence B*: `ACCGTGGCA`

Can you spot the differences? Would you be able to align both sequences in such a way that we can easily pinpoint the matches and mismatches? Something like what you get if you press the following Run button:

In [63]:
def test_aligner():
    seq_a = 'ACTGTCGCA'
    seq_b = 'ACCGTGGCA'
    aligner = Align.PairwiseAligner()

    aligner.mode = 'global'     
    aligner.match_score = 1      
    aligner.mismatch_score = 1     
    aligner.open_gap_score = 0.1
    aligner.extend_gap_score = 0.1      

    alignments = aligner.align(seq_a, seq_b)
    print(f'Alignment between the sequences:\n')
    print(alignments[0])

test_aligner()

Alignment between the sequences:

ACTGTCGCA
||.||.|||
ACCGTGGCA



I'm sure you do, but bioinformatic pipelines routinely deal with sequences thousand of bases long. How can we mathematically align two sequences like this in a consistent and reproducible way? Enter the alignment matrix:


|   | - | A | C | T | G | T | C | G | C | A  |
|---|---|---|---|---|---|---|---|---|---|---|
| - | 0 |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| T |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |

Creating an alignment score matrix is a foundational step in understanding sequence alignment algorithms. The matrix helps visualize the process of aligning two sequences by scoring similarities, differences, and gaps. For this explanation, we'll focus on a simple scoring scheme: +1 for a match, -1 for a mismatch, and -2 for a gap. The goal is to align these sequences to maximize the alignment score based on our scoring scheme. Our very first value (*i0*, *j0*) is a 0, representing the cost of missaligning two empty sequences.

We'll start by initializing our first row with gap penalties - the cost of skipping a base to better align the sequences. This step is necessary to begin filling the matrix.

|   | - | A | C | T | G | T | C | G | C | A  |
|---|---|---|---|---|---|---|---|---|---|---|
| - | 0 | -2| -4| -6| -8| -10| -12| -14| -16| -18|
| A |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| T |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| G |   |   |   |   |   |   |   |   |   |   |
| C |   |   |   |   |   |   |   |   |   |   |
| A |   |   |   |   |   |   |   |   |   |   |

Then, the rules of the game are simple. Starting from the first position:
1. If the characters match, add the match score (+1) to the upper-left diagonal value.
2. If the characters don't match, take the maximum of:
    - The diagonal value plus the mismatch score (-1).
    - The left value plus the gap penalty (-2).
    - The above value plus the gap penalty (-2).
3. Repeat step 2 for each cell in the matrix until the last value (*i<sub>m</sub>*, *j<sub>n</sub>*).

You should obtain a table like this (don't worry, you don't have to do it manually)


|   | - | A  | C  | T  | G  | T  | C  | G  | C  | A  |
|---|---|----|----|----|----|----|----|----|----|----|
| - | 0 | -2 | -4 | -6 | -8 | -10| -12| -14| -16| -18|
| A | -2| 1  | -1 | -3 | -5 | -7 | -9 | -11| -13| -15|
| C | -4| -1 | 2  | 0  | -2 | -4 | -6 | -8 | -10| -12|
| C | -6| -3 | 0  | 1  | -1 | -1 | -3 | -5 | -7 | -9 |
| G | -8| -5 | -2 | -1 | 2  | 0  | 0  | -2 | -4 | -6 |
| T |-10| -7 | -4 | -3 | 0  | 3  | 1  | -1 | -3 | -5 |
| G |-12| -9 | -6 | -3 | -2 | 1  | 2  | 0  | 0  | -2 |
| G |-14|-11 | -8 | -5 | -2 | -1 | 2  | 3  | 1  | -1 |
| C |-16|-13 | -10| -7 | -4 | -3 | 0  | 1  | 4  | 2  |
| A |-18|-15 | -12| -9 | -6 | -5 | -2 | -1 | 2  | 5  |


And now that this is over, we would need to **reconstruct** our path based on the choices we made (diagonal, left or up) in a process called *traceback*. First we must choose the right cell to begin this *traceback*:

- For global alignment ([Needleman-Wunsch](https://www.wikiwand.com/en/Needleman%E2%80%93Wunsch_algorithm)), start from the bottom-right cell of the matrix.
- For local alignment ([Smith-Waterman](https://www.wikiwand.com/en/Smith%E2%80%93Waterman_algorithm)), start from the highest-scoring cell in the entire matrix.

*Traceback* Steps:
- If the optimal path moves diagonally (from (*i*, *j*) to (*i-1*, *j-1*)), it indicates a match or mismatch. Add the corresponding characters from both sequences to the alignment and move to the cell (*i-1*, *j-1*).
- If the optimal path moves up (from (*i*, *j*) to (*i-1*, *j*)), it indicates a gap in the second sequence. Add a gap character (-) to the alignment for the second sequence, the character from the first sequence, and move to the cell (*i-1*, *j*).
- If the optimal path moves left (from (*i*, *j*) to (*i*, *j-1*)), it indicates a gap in the first sequence. Add a gap character (-) to the alignment for the first sequence, the character from the second sequence, and move to the cell (*i*, *j*) to (*i*, *j-1*).

Repeat the process of checking for diagonal, up, or left moves until you reach the finishing cell:
- For global alignment, this is the top-left cell (*i0*, *j0*).
- For local alignment, this is any cell on the border where the traceback started (depending on the algorithm's rules, you might stop once you hit a score of 0 to only include the high-scoring local region).

We can generate a programmatic implementation of the global algorithm  with a few lines of code. Press the Run button to see how it performs.

In [64]:
def needleman_wunsch_test():
    # Initialize the scoring matrix
    seq1 = 'ACTGTCGCA'
    seq2 = 'ACCGTGGCA'
    match_score = 1
    mismatch_penalty = -1
    gap_penalty = -2
    m, n = len(seq1), len(seq2)
    score_matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]

    # Initialize the edges with gap penalties
    for i in range(m + 1):
        score_matrix[i][0] = i * gap_penalty
    for j in range(n + 1):
        score_matrix[0][j] = j * gap_penalty

    # Fill in the scoring matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

    print(pd.DataFrame(score_matrix, columns = ['-'] + list(seq1), index = ['-'] + list(seq2)))

needleman_wunsch_test()


    -   A   C  T  G   T   C   G   C   A
-   0  -2  -4 -6 -8 -10 -12 -14 -16 -18
A  -2   1  -1 -3 -5  -7  -9 -11 -13 -15
C  -4  -1   2  0 -2  -4  -6  -8 -10 -12
C  -6  -3   0  1 -1  -1  -3  -5  -7  -9
G  -8  -5  -2 -1  2   0   0  -2  -4  -6
T -10  -7  -4 -3  0   3   1  -1  -3  -5
G -12  -9  -6 -3 -2   1   2   0   0  -2
G -14 -11  -8 -5 -2  -1   2   3   1  -1
C -16 -13 -10 -7 -4  -3   0   1   4   2
A -18 -15 -12 -9 -6  -5  -2  -1   2   5


Few! What a pain! Fortunately, the Biopython library contains modules that are here to take the burden out of it. Sit back, take a breather and let the program do it in a second pressing the Run button. You have earned it.

In [65]:
def entrez_aligner():
    sequence_info = seq_reader()
    print('Aligning sequences...')
    aligner = Align.PairwiseAligner()

    aligner.mode = 'local'     # Choose "global" or "local"
    aligner.match_score = 1      # Choose your match score here, for example -3
    aligner.mismatch_score = 1     # Choose your mismatch score here, for example 3
    aligner.open_gap_score = 0.1     # Choose your gap penalties here, for example 0.6
    aligner.extend_gap_score = 0.1      # Choose your gap penalties here, for example 0.6

    alignments = aligner.align(sequence_info[0].seq, sequence_info[1].seq)
    print(alignments[0])

entrez_aligner()

2 sequences read.
Aligning sequences...
GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATGTACCAGAACAACCCGCTGTCGGGGTTGACCCACAAGCGCCGACTGTCGGCGCTGGGGCCCGGCGGTCTGTCACGTGAGCGTGCCGGGCTG
||||||||||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||||.||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGCCAATTCATGGACCAGAACAACCCGTTGTCGGGGTTGACCCACAAGCGCCGACTGTCGGCGCTGGGGCCCGGCGGTCTGTCACGTGAGCGTGCCGGGCTG



Now that's a much better way to visualize an alignment. The midline is composed of three symbols:
- "|" represents a match.
- "-" represents a gap.
- "." represents a substitution.

Feel free to experiment with different scoring parameters and alignment modes. For reference:
- Global mode focuses on maximizing the alignment from end to end of both sequences.
- Local mode focuses on maximizing the alignment in the most similar regions of both sequences.

Observe how modifying the different scores alters the alignment output. As a quick summary, remember the following points:
1. We use pairwise alignment algorithms to obtain the best matching score between our sequences given our scoring scheme.
2. We fill the matrix sequentially based on a set of instructions.
3. Then we construct the alignment by *tracing back* our steps. Depending on our choices we write bases or gaps until our alignment is complete.

*By the way, imagine doing the above steps manually with sequences of hundreds or even [thousands](https://www.wikiwand.com/en/Masochism_(disambiguation)) of bases.*

Now that's out of the way, we will learn how to perform a [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi). BLAST (Basic Local Alignment Search Tool) is a fundamental algorithm for comparing an individual biological sequence (such as amino acids for proteins or nucleotides for DNA/RNA) with a library or database of sequences, and it works with the exact same principles of alignment you just learnt in the previous step.

BLAST works as following:
1. Query Sequence: The user submits a sequence to BLAST, which serves as the query.
2. Database Search: BLAST compares the query sequence against a database of sequences to find matches.
3. Identify Matches: It identifies high-scoring sequence pairs (HSPs) by looking for word matches between the query and database sequences and then extending these matches to find longer alignments without significant gaps.
4. Scoring: Each potential alignment is scored based on the number of matches, mismatches, and gaps, with higher scores indicating better alignments.
5. Results: BLAST returns a list of database sequences that are similar to the query sequence, including alignments and statistical measures of significance (such as [E-values](https://www.wikiwand.com/en/E-values), which estimate the number of chance matches in a database of a given size).

For a visual, nice refresher into alignments and BLAST, check this very nice article courtesy of [Nature](https://www.nature.com/scitable/topicpage/basic-local-alignment-search-tool-blast-29096/).

To continue with the exercise, we'll pick one of the two sequences at random and BLAST it. Enough with the talk! Hit that Run button and let the computer handle the rest.

In [66]:
def blast_sequence_individual():
    sequences = list(SeqIO.parse("filtered_sequences.fasta", "fasta"))

    print(f'BLASTing sequence {sequences[0].id}...')
    result_handle = NCBIWWW.qblast("blastn", 
                                    "nt", 
                                    sequences[0].seq,
                                    expect=0.001,
                                    hitlist_size=100)
                                    
    with open(f"blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()
    print(f'Successfully BLASTed sequence {sequences[0].id}.')

def blast_sequence_individual_sim():
    sequences = list(SeqIO.parse("filtered_sequences.fasta", "fasta"))
    print(f"BLASTing sequence {sequences[0].id}...")
    time.sleep(5)
    print(f'Successfully BLASTed sequence {sequences[0].id}.')
            

blast_sequence_individual_sim()

BLASTing sequence LN651305.1...


Successfully BLASTed sequence LN651305.1.


Aaaand another simulacrum. The reasons for doing so are pretty much the same as before. Imagine having to calculate all those awful alignment matrices, but against millions of sequences, looking for the ones that return the best statistical probability of a non-random match. Talk about finding a needle in a genestack! In fact, the [genius](https://blastalgorithm.com/) of the BLAST database search is the speed and efficiency with which the algorithm handles such a colosal hunt. Nonetheless, if you run the non-simulacrum function, expect to wait for a few long minutes.

Our BLAST has been executed. Feel free to explore the file called "blast_result.xml" by clicking on it. You will see that there is a pattern repeating itself within the document, and what appear to be metrics and sequences. These are the results returned from the database, and now we have to parse them nicely. Press the Run button to forget about the how-tos.

In [67]:
def parse_blast_results_to_fasta():
    hsps = []
    with open(f'blast_result.xml') as result_file:
        blast_record = NCBIXML.read(result_file)
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                hsps.append(hsp)
    
    print(f'Parsed {len(hsps)} HSPs. Check file "aggregated_sequences.fasta"')
    print(f'First HSP: {hsps[0]}.')

parse_blast_results_to_fasta()

Parsed 100 HSPs. Check file "aggregated_sequences.fasta"
First HSP: Score 288 (260 bits), expectation 4.7e-65, alignment length 144
Query:       1 GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGC...CTG 144
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:  757255 GTGGTCGCCGCGATCAAGGAGTTCTTCGGCACCAGCCAGCTGAGC...CTG 757398.


One hundred HSPs! That means that from our original query, the BLAST has found one hundred matching sequences and has calculated the metrics that allow us to quantitatively measure the degree of similarity and length between them.

One hundred sequences. Would it be possible to align **all** of them? Would it be neccessary to perform all possible pairwise alignments between all possible pairs of sequences? That would be nothing less than 100<sup>2</sup> = 10000 pairwise alignments. And they wouldn't account for a combined alignment score, but only for individual pairs. For this, we need a new mathematical tool: Multiple Sequence Alignment - don't worry, we won't enter into the gritty mathematical details. Press the Run button to perform another simulacrum.

In [68]:
def clustal_msa_aligner():
    print('Aligning sequences with ClustalOmega...')
    clustalomega_cline = ClustalOmegaCommandline(infile="aggregated_sequences.fasta", outfile="aligned_sequences.fasta", verbose=True, auto=True, force=True)
    stdout, stderr = clustalomega_cline()
    print(f'Multiple Sequence Alignment Complete. Check file "aligned_sequences.fasta"')

def clustal_msa_aligner_sim():
    print('Aligning sequences with ClustalOmega...')
    time.sleep(5)
    print(f'Multiple Sequence Alignment Complete. Check file "aligned_sequences.fasta"')

clustal_msa_aligner_sim()


Aligning sequences with ClustalOmega...


Multiple Sequence Alignment Complete. Check file "aligned_sequences.fasta"


At this point you may be thinking that half of this code doesn't work in a real setting. Too many simulacrums. Well, in this case it's because we are running an external tool to the environment. If pairwise alignment looks tedious enough from the mathematical standpoint, it is Disney World in comparison to the complexity of performing MSA, and it can be easily verified by the amount of time and resources it takes to perform a simple one between a moderate amount of not very long sequences. However, if you want to have a second opinion, follow these instructions:

1. Click the following [link](https://www.ebi.ac.uk/jdispatcher/msa/clustalo).
2. Select "DNA".
3. Click "Choose File" and select "aggregated_sequences.fasta".
4. Write a title for the job.
5. Click submit.
6. Sit back and enjoy while a high-performance cluster of computers at the other side of the Atlantic churns the numbers.

OR

1. Check the following [link](https://www.ebi.ac.uk/Tools/services/rest/mview/result/mview-I20240318-212727-0482-15810022-p1m/aln-html) for a precomputed solution in HTML.

However you choose to do it, the result will be an output file you already have available: "aligned_sequences.fasta". It will be the last ingredient we need for what lies ahead. Press that Run button, we are almost finished!


In [69]:
def distance_matrix_calculator():
    print('Calculating Distance Matrix...')
    align = AlignIO.read("aligned_sequences.fasta", "fasta")
    calculator = DistanceCalculator('identity') # Try other options like "blastn", "trans", "benner6", "benner22", "benner74"
    dm = calculator.get_distance(align)

    # Convert the identity matrix to a pandas DataFrame for nicer output
    split_names = [_.split("|")[-1] for _ in dm.names]
    df = pd.DataFrame(dm.matrix, columns=split_names, index=split_names)
    print('Identity Matrix Sample:')
    print(df.iloc[:7, :7])

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
distance_matrix_calculator()

Calculating Distance Matrix...
Identity Matrix Sample:
                                                                 
  0.000000       NaN       NaN       NaN       NaN       NaN  NaN
  0.041667  0.000000       NaN       NaN       NaN       NaN  NaN
  0.041667  0.041667  0.000000       NaN       NaN       NaN  NaN
  0.041667  0.041667  0.041667  0.000000       NaN       NaN  NaN
  0.041667  0.041667  0.041667  0.041667  0.000000       NaN  NaN
  0.041667  0.041667  0.041667  0.041667  0.041667  0.000000  NaN
  0.041667  0.041667  0.041667  0.041667  0.041667  0.041667  0.0


Our algorithms have generated a [distance matrix](https://www.wikiwand.com/en/Distance_matrix). This is a diagonal, squared matrix that contains the "distances" between all pairs of our already aligned sequences. The diagonal is composed of 0s, because there is no "distance" between a sequence and itself. Feel free to experiment with the options in the function and see how they change the scores in the sample output matrix. When you are ready, press the Run button one final time.

In [70]:
def phylogeny_calculator():
    print('Constructing Phylogenetic Tree...')
    align = AlignIO.read("aligned_sequences.fasta", "fasta")
    calculator = DistanceCalculator('identity') # Try other options like "blastn", "trans", "benner6", "benner22", "benner74"
    dm = calculator.get_distance(align)
    constructor = DistanceTreeConstructor(calculator, 'nj')  # Neighbor-Joining method. Try "upgma" (Unweighted Pair Group Method with Arithmetic Mean)
    tree = constructor.build_tree(align)
    Phylo.write(tree, "phylogeny.xml", "phyloxml")

    # Optionally, display the tree
    Phylo.draw_ascii(tree)
    
phylogeny_calculator()

Constructing Phylogenetic Tree...
          _________________ gi|2556671665|gb|CP130772.1|
    _____|
  _|     |_________________ gi|2556671679|gb|CP130788.1|
 | |
 | |_______________________ gi|2607448098|gb|CP097111.1|
 |
 |    _____________________ gi|2556671736|gb|CP130805.1|
 |___|
 |   |_____________________ gi|2646972455|dbj|AP024456.1|
 |
 |        __________________ gi|2646996763|dbj|AP024462.1|
 |  _____|
 | |     |_______________________ gi|2240006232|gb|OL961725.1|
 | |
 | |      _________________ gi|2556671685|gb|CP130795.1|
 | |_____|
 | |     |_________________ gi|309753193|gb|HQ286624.1|
 | |
 | |        _________________ gi|2556671741|gb|CP130811.1|
 | |      _|
 | |  ___| |_________________ gi|2556671747|gb|CP130815.1|
 | | |   |
 | | |   |______________ gi|669073577|gb|CP008969.1|
 | | |
 | | |   _________________________ gi|1253806357|gb|MF981086.1|
 | | |  |
 | | |  |    ______________________ gi|554681129|gb|KF444081.1|
 | | |  |  _|
 |,| |  | | |    _____________

Last but not least, here's the promised [phylogenetic tree](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7123334/). Our functions have lead us to a point in which we can compare all the sequences with mathematical accuracy and draw such a tree. The distribution of the branches tells us about the possible common ancestors of each [genomic variant](https://www.genome.gov/about-genomics/educational-resources/fact-sheets/human-genomic-variation), while the length of the branches represent the figurative distance between the sequences. Feel free to once again modify the options in the constructor function, and see how they change the shape of the tree. 

This tree informs us about the proximity of the sequences analysed, and it's a good starting point to cluster our variants in an informative way. 

Congratulations on having arrived to the end the lesson. These are some of the basic tools bioinformaticians across the world use routinely in the fight agains TB; but as the saying goes, the rabbit hole goes much, much deeper. Huge international research efforts are both making use of these technologies, as well as trying to improve upon them. As a closure to this exercise, I invite you to take a look at some of the most fascinating current projects that involve technologies like these on a daily basis:
- [The Earth BioGenome Project (EBP)](https://www.earthbiogenome.org/), aims to sequence, catalog, and characterize the genomes of all of Earth's eukaryotic biodiversity over a period of 10 years.
- [The ENCODE Project](https://www.encodeproject.org/) aims to annotate the full human genome. That is, not only identifying the DNA sequence and its variations, but what each part of the sequence does. [The 1000 Genomes Project](https://www.internationalgenome.org/home) remains one of the largest repository of genomic human variation data, and it is heavily used by the biomedical community. Both are spiritual successors of the [Human Genome Project](https://www.genome.gov/human-genome-project), often considered the Manhattan Project of biology.
- [AGAThA (GPU Acceleration of Guided Sequence Alignment for Long Read Mapping)](https://arxiv.org/abs/2403.06478) aims to take sequence alignment to the next level by enabling GPU parallel architectures into alignment algorithms, while [Embed-Search-Align using Transformer Models](https://ar5iv.labs.arxiv.org/html/2309.11087) takes inspiration in the similarities between DNA sequences and human languages, using technology originally developed for natural language processing (NLP).

If you want to delve further into the science and art of biological data management and analysis, I recommend the following resources (I'm not affiliated in any way to them):
- Molecular Biology of the Cell, by Alberts et al., the renouned reference manual for all things molecular, already in its 7th edition.
- [The Biostars Handbook](https://www.biostarhandbook.com/), a beginner-friendly approach to all things bioinformatics, from scripts to Next Generation Sequencing.
- [From Cell Line to Command Line](https://divingintogeneticsandgenomics.ck.page/), for those biologists that want to make the jump into the computational side of things.

I hope you liked this little exercise, and that you can take something from it back to your professional and daily life! Please don't hesitate to make any questions, and I hope to hear from you all soon!